In [1]:
import pandas as pd
import plotly.express as px
import requests
from xml.etree import ElementTree as ET

import plotly.graph_objects as go
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px



%load_ext google.colab.data_table


In [2]:
df0 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Work_LCHS/160JourneyDataExtract01May2019-07May2019.csv')
df1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Work_LCHS/161JourneyDataExtract08May2019-14May2019.csv')
df2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Work_LCHS/162JourneyDataExtract15May2019-21May2019.csv')
df3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Work_LCHS/163JourneyDataExtract22May2019-28May2019.csv')
df4 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Work_LCHS/164JourneyDataExtract29May2019-04Jun2019.csv')

In [3]:
frames = [df0,df1,df2,df3,df4]

In [4]:
df = pd.concat(frames)

In [5]:
df.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,86539944,180,16202,05/05/2019 10:11,446,"York Hall, Bethnal Green",05/05/2019 10:08,719,"Victoria Park Road, Hackney Central"
1,86585923,240,14673,07/05/2019 07:24,578,"Hollybush Gardens, Bethnal Green",07/05/2019 07:20,719,"Victoria Park Road, Hackney Central"
2,86609010,780,10885,07/05/2019 18:02,40,"Commercial Street, Shoreditch",07/05/2019 17:49,719,"Victoria Park Road, Hackney Central"
3,86563700,240,7782,06/05/2019 10:03,39,"Shoreditch High Street, Shoreditch",06/05/2019 09:59,702,"Durant Street, Bethnal Green"
4,86498870,300,7846,03/05/2019 11:17,39,"Shoreditch High Street, Shoreditch",03/05/2019 11:12,702,"Durant Street, Bethnal Green"


In [6]:
site = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"

response = requests.get(site)
root = ET.fromstring(response.content)

id_list = [int(root[i][0].text) for i in range(0, len(root))]
name_list = [root[i][1].text for i in range(0, len(root))]
lat_list = [float(root[i][3].text) for i in range(0, len(root))]
lon_list = [float(root[i][4].text) for i in range(0, len(root))]
capacity_list = [int(root[i][12].text) for i in range(0, len(root))]

all_locs = pd.DataFrame(list(zip(name_list, id_list, lat_list, 
                                 lon_list, capacity_list)), columns = ["name","id","lat","lon","capacity"])

#all_locs.to_csv('bike_point_locations_saved.csv', header=True, index=None)

#print(all_locs.shape)
#all_locs.head()

In [7]:
df_loc = df.merge(all_locs, how='left', left_on=['EndStation Name'], right_on=['name'])
df_loc.drop(['name','id'], axis=1, inplace=True)
df_loc.rename(columns={'lat':'end_lat','lon':'end_lon', 'capacity':'capacity_end'}, inplace=True)
df_loc = df_loc.merge(all_locs, how='left', left_on=['StartStation Name'], right_on=['name'])
df_loc.drop(['name','id'], axis=1, inplace=True)
df_loc.rename(columns={'lat':'start_lat','lon':'start_lon', 'capacity':'capacity_start'}, inplace=True)

df_loc.drop(columns=['EndStation Id','StartStation Id'], inplace=True)


df_loc.head()


,Rental Id,Duration,Bike Id,End Date,EndStation Name,Start Date,StartStation Name,end_lat,end_lon,capacity_end,start_lat,start_lon,capacity_start
0,86539944,180,16202,05/05/2019 10:11,"York Hall, Bethnal Green",05/05/2019 10:08,"Victoria Park Road, Hackney Central",51.528936,-0.055894,27.0,51.536425,-0.054162,34.0
1,86585923,240,14673,07/05/2019 07:24,"Hollybush Gardens, Bethnal Green",07/05/2019 07:20,"Victoria Park Road, Hackney Central",51.527607,-0.057133,32.0,51.536425,-0.054162,34.0
2,86609010,780,10885,07/05/2019 18:02,"Commercial Street, Shoreditch",07/05/2019 17:49,"Victoria Park Road, Hackney Central",51.521271,-0.075579,16.0,51.536425,-0.054162,34.0
3,86563700,240,7782,06/05/2019 10:03,"Shoreditch High Street, Shoreditch",06/05/2019 09:59,"Durant Street, Bethnal Green",51.526378,-0.078131,41.0,51.528682,-0.065550,24.0
4,86498870,300,7846,03/05/2019 11:17,"Shoreditch High Street, Shoreditch",03/05/2019 11:12,"Durant Street, Bethnal Green",51.526378,-0.078131,41.0,51.528682,-0.065550,24.0


In [8]:
#df_loc.columns

In [9]:
new_order = ['Rental Id', 'Duration', 'Bike Id', 
       'Start Date', 'StartStation Name','start_lat',
        'start_lon','capacity_start','End Date', 'EndStation Name', 'end_lat', 'end_lon', 'capacity_end' ]

In [10]:
df_loc = df_loc.reindex(columns=new_order)

In [11]:
df_loc['End Date'] = pd.to_datetime(df_loc['End Date'], format='%d/%m/%Y %H:%M')
df_loc['Start Date'] = pd.to_datetime(df_loc['Start Date'], format='%d/%m/%Y %H:%M')
df_loc['hire_day'] = df_loc['Start Date'].dt.day_name()

In [12]:
villages = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Work_LCHS/villages.csv', usecols=['DESCRIPTION', 'VILLAGE'])

df_loc = df_loc.merge(villages, how='left', left_on=['StartStation Name'], right_on='DESCRIPTION').drop('DESCRIPTION', axis=1)
df_loc = df_loc.merge(villages, how='left', left_on=['EndStation Name'], right_on='DESCRIPTION').drop('DESCRIPTION', axis=1)
df_loc.rename(columns={'VILLAGE_x':'village_start','VILLAGE_y':'village_end'}, inplace=True)

In [13]:
df_loc.head()

,Rental Id,Duration,Bike Id,Start Date,StartStation Name,start_lat,start_lon,capacity_start,End Date,EndStation Name,end_lat,end_lon,capacity_end,hire_day,village_start,village_end
0,86539944,180,16202,2019-05-05 10:08:00,"Victoria Park Road, Hackney Central",51.536425,-0.054162,34.0,2019-05-05 10:11:00,"York Hall, Bethnal Green",51.528936,-0.055894,27.0,Sunday,F1,F1
1,86585923,240,14673,2019-05-07 07:20:00,"Victoria Park Road, Hackney Central",51.536425,-0.054162,34.0,2019-05-07 07:24:00,"Hollybush Gardens, Bethnal Green",51.527607,-0.057133,32.0,Tuesday,F1,F1
2,86609010,780,10885,2019-05-07 17:49:00,"Victoria Park Road, Hackney Central",51.536425,-0.054162,34.0,2019-05-07 18:02:00,"Commercial Street, Shoreditch",51.521271,-0.075579,16.0,Tuesday,F1,A4
3,86563700,240,7782,2019-05-06 09:59:00,"Durant Street, Bethnal Green",51.528682,-0.065550,24.0,2019-05-06 10:03:00,"Shoreditch High Street, Shoreditch",51.526378,-0.078131,41.0,Monday,F1,A4
4,86498870,300,7846,2019-05-03 11:12:00,"Durant Street, Bethnal Green",51.528682,-0.065550,24.0,2019-05-03 11:17:00,"Shoreditch High Street, Shoreditch",51.526378,-0.078131,41.0,Friday,F1,A4


In [14]:
trips_village = pd.DataFrame(df_loc.groupby('village_start')['village_end'].value_counts())

In [15]:
#trips_village.reset_index(level=0, inplace=True)
trips_village.rename(columns={'village_end':'count'}, inplace=True)
#trips_village.set_index('village_start', inplace=True)
#trips_village.rename(columns={trips_village.columns[2]:'village_end'})

In [16]:
trips_village.reset_index(inplace=True)
trips_village.head()

,village_start,village_end,count
0,A1,A1,7385
1,A1,E2,4922
2,A1,E3,4451
3,A1,D2,4349
4,A1,A4,3445


In [17]:
trips_village_ex = trips_village.query('village_start != village_end')

In [18]:
#trips_village_ex.set_index('village_start', inplace=True)

In [19]:
source = [0] * len(trips_village_ex)

In [20]:
trips_village_ex['source'] = source

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
trips_village_ex.head()

,village_start,village_end,count,source
1,A1,E2,4922,0
2,A1,E3,4451,0
3,A1,D2,4349,0
4,A1,A4,3445,0
5,A1,A2,2841,0


In [22]:
trips_a1 = trips_village_ex[trips_village_ex.village_start == 'A1']

In [23]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#FFA07A', '#FFA500','#ADFF2F','#008B8B','#D2691E','#2F4F4F']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [24]:
fig = genSankey(trips_a1,cat_cols=['village_start','village_end'],value_cols='count',title='Trips from A1')
py.offline.plot(fig, validate=False)


'temp-plot.html'

In [25]:
"""data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 10,
      thickness = 30,
      line = dict(
        color = "black",
        width = 0
      ),
      label =  trips_village_ex['village_end'],
      #color = trips_village_ex['Color']
    ),
    link = dict(
      source = trips_village_ex['source'],
      target = trips_village_ex['count'],
      value = trips_village_ex['count'],
      #color = trips_village_ex['Link Color'],
  )
)

layout =  dict(
    title = "Prova",
    height = 772,
    font = dict(
      size = 10
    ),    
)

fig = dict(data=[data_trace], layout=layout)
iplot(fig, validate=False)"""


'data_trace = dict(\n    type=\'sankey\',\n    domain = dict(\n      x =  [0,1],\n      y =  [0,1]\n    ),\n    orientation = "h",\n    valueformat = ".0f",\n    node = dict(\n      pad = 10,\n      thickness = 30,\n      line = dict(\n        color = "black",\n        width = 0\n      ),\n      label =  trips_village_ex[\'village_end\'],\n      #color = trips_village_ex[\'Color\']\n    ),\n    link = dict(\n      source = trips_village_ex[\'source\'],\n      target = trips_village_ex[\'count\'],\n      value = trips_village_ex[\'count\'],\n      #color = trips_village_ex[\'Link Color\'],\n  )\n)\n\nlayout =  dict(\n    title = "Prova",\n    height = 772,\n    font = dict(\n      size = 10\n    ),    \n)\n\nfig = dict(data=[data_trace], layout=layout)\niplot(fig, validate=False)'

## Analysis by Station

In [26]:
trips = pd.DataFrame(df_loc.groupby(['StartStation Name'])['EndStation Name'].value_counts()).reset_index(level=0)

In [27]:
trips.rename(columns={'EndStation Name':'count'}, inplace=True)

In [28]:
trips.head()

,StartStation Name,count
EndStation Name,,
"Belgrove Street , King's Cross","Abbey Orchard Street, Westminster",77
"Waterloo Station 3, Waterloo","Abbey Orchard Street, Westminster",43
"Abbey Orchard Street, Westminster","Abbey Orchard Street, Westminster",41
"Hyde Park Corner, Hyde Park","Abbey Orchard Street, Westminster",30
"Kennington Lane Rail Bridge, Vauxhall","Abbey Orchard Street, Westminster",28


In [29]:
trips.sort_values(by='count', ascending=False)

,StartStation Name,count
EndStation Name,,
"Hyde Park Corner, Hyde Park","Hyde Park Corner, Hyde Park",2343
"Aquatic Centre, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",2307
"Black Lion Gate, Kensington Gardens","Black Lion Gate, Kensington Gardens",1312
"Albert Gate, Hyde Park","Albert Gate, Hyde Park",890
"Park Lane , Hyde Park","Park Lane , Hyde Park",806
...,...,...
"Sugden Road, Clapham","Imperial Road, Sands End",1
"Star Road, West Kensington","Imperial Road, Sands End",1
"St. Luke's Church, Chelsea","Imperial Road, Sands End",1


In [30]:
trips[trips['StartStation Name'].str.contains('Aquatic')]

,StartStation Name,count
EndStation Name,,
"Aquatic Centre, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",2307
"Monier Road, Hackney Wick","Aquatic Centre, Queen Elizabeth Olympic Park",412
"Lee Valley VeloPark, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",375
"Podium, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",333
"Timber Lodge, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",244
...,...,...
"Wenlock Road , Hoxton","Aquatic Centre, Queen Elizabeth Olympic Park",1
"Whitehall Place, Strand","Aquatic Centre, Queen Elizabeth Olympic Park",1
"Wormwood Street, Liverpool Street","Aquatic Centre, Queen Elizabeth Olympic Park",1


In [31]:
trips[trips['StartStation Name'] == trips.index]

,StartStation Name,count
EndStation Name,,
"Abbey Orchard Street, Westminster","Abbey Orchard Street, Westminster",41
"Abbotsbury Road, Holland Park","Abbotsbury Road, Holland Park",32
"Aberdeen Place, St. John's Wood","Aberdeen Place, St. John's Wood",15
"Aberfeldy Street, Poplar","Aberfeldy Street, Poplar",20
"Abingdon Green, Westminster","Abingdon Green, Westminster",52
...,...,...
"Wren Street, Holborn","Wren Street, Holborn",28
"Wright's Lane, Kensington","Wright's Lane, Kensington",51
"Wynne Road, Stockwell","Wynne Road, Stockwell",16


In [32]:
trips[['Hyde' in s for s in trips.index] & trips['StartStation Name'].str.contains('Hyde')]


,StartStation Name,count
EndStation Name,,
"Albert Gate, Hyde Park","Albert Gate, Hyde Park",890
"Hyde Park Corner, Hyde Park","Albert Gate, Hyde Park",488
"Serpentine Car Park, Hyde Park","Albert Gate, Hyde Park",238
"Triangle Car Park, Hyde Park","Albert Gate, Hyde Park",231
"Park Lane , Hyde Park","Albert Gate, Hyde Park",198
...,...,...
"Serpentine Car Park, Hyde Park","Wellington Arch, Hyde Park",145
"Albert Gate, Hyde Park","Wellington Arch, Hyde Park",106
"Bayswater Road, Hyde Park","Wellington Arch, Hyde Park",74


In [33]:
trips[['Queen Elizabeth Olympic Park' in s for s in trips.index] & trips['StartStation Name'].str.contains('Queen Elizabeth Olympic Park')]


,StartStation Name,count
EndStation Name,,
"Aquatic Centre, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",2307
"Lee Valley VeloPark, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",375
"Podium, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",333
"Timber Lodge, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",244
"Copper Box Arena, Queen Elizabeth Olympic Park","Aquatic Centre, Queen Elizabeth Olympic Park",127
...,...,...
"East Village, Queen Elizabeth Olympic Park","Timber Lodge, Queen Elizabeth Olympic Park",110
"Podium, Queen Elizabeth Olympic Park","Timber Lodge, Queen Elizabeth Olympic Park",110
"Copper Box Arena, Queen Elizabeth Olympic Park","Timber Lodge, Queen Elizabeth Olympic Park",45


In [34]:
trips.reset_index()

,EndStation Name,StartStation Name,count
0,"Belgrove Street , King's Cross","Abbey Orchard Street, Westminster",77
1,"Waterloo Station 3, Waterloo","Abbey Orchard Street, Westminster",43
2,"Abbey Orchard Street, Westminster","Abbey Orchard Street, Westminster",41
3,"Hyde Park Corner, Hyde Park","Abbey Orchard Street, Westminster",30
4,"Kennington Lane Rail Bridge, Vauxhall","Abbey Orchard Street, Westminster",28
...,...,...,...
204985,"Waterloo Station 3, Waterloo","York Way, Kings Cross",1
204986,"Wellington Arch, Hyde Park","York Way, Kings Cross",1
204987,"Wenlock Road , Hoxton","York Way, Kings Cross",1
204988,"Whiston Road, Haggerston","York Way, Kings Cross",1


In [35]:
! df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   31G   73G  30% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.9G     0  5.9G   0% /dev/shm
tmpfs           6.4G   12K  6.4G   1% /var/colab
/dev/sda1       114G   32G   83G  28% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G  7.1G  8.0G  47% /content/drive
